## Septante nuances de swisscovery

In [3]:
import ipywidgets as widgets
from IPython.display import display, clear_output
from langchain_community.llms import LlamaCpp
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import multiprocessing
from datetime import datetime
import re
import urllib.parse
import json
import requests
import warnings
warnings.filterwarnings('ignore')

# Paramètres
swisscovery_api = 'https://api-eu.hosted.exlibrisgroup.com/primo/v1/search?vid=41SLSP_UGE:VU1&tab=CentralIndex'
swisscovery_key = 'xxx' # Sandbox

# Recherche pour les tests 
# recherche = "Je veux savoir ce qu'est le bosson de Higgs"

flex_subtitle = "built using jupyter-flex"
flex_external_link = "https://github.com/dis-unige/hackademia-2024/tests/test_swisscoveryIA.ipynb"

flex_title = "swisscoveryIA"
flex_show_source = True

### Pose ta question en langage naturel pour explorer la collection de la Bibliothèque de l'UNIGE

In [ ]:
def on_button_clicked2(event):
    print("TEST OK")
    with output:
        clear_output()
        print(f"Recherche lancée, veuillez patienter, l'IA est un peu lente... {search.value}")

In [ ]:
# function de chat
def chat(question):
    # Load the LlamaCpp language model, adjust GPU usage based on your hardware
    llm = LlamaCpp(
        model_path="C:/Users/pablo/AI/llms/llama-2-7b-chat.Q4_K_M.gguf",
        temperature=0.5,
        n_ctx=10000,
        n_gpu_layers=-1,
        n_batch=9900,  # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
        max_tokens=512,
        n_threads=multiprocessing.cpu_count() - 1,
        ## repeat_penalty=1.5,
        # top_p=0.5,
        verbose=False,  # Enable detailed logging for debugging
    )
    
    # Define the prompt template with a placeholder for the question
    template = """
    Question: I want an concise explanation about the question "{question}" and also to transform the question into a booleen search strategy that can be used in a library catalog.
    The search query can be constructed using several synonyms of the subject in french and english. All the terms of the query can be combined with OR.
    Put the string '[explanation]' before your explanation about the question.
    Put the string '[query]' before que search query.
    Don't add comments inside the query, give only the serach strategy the we can use in our library database.
    Please respect this structure on the answer: '[explanation] the subject explanation. [query] the search strategy' and don't add explanations after the query
    
    Answer:
    """
    prompt = PromptTemplate(template=template, input_variables=["question"])
    
    # Create an LLMChain to manage interactions with the prompt and model
    llm_chain = LLMChain(prompt=prompt, llm=llm)
    
    # print("Chatbot initialized, ready to chat...")
    while True:
        # question = input("> ")
        # print('start IA : ' + str(datetime.now())) 
        # answer = llm_chain.run(question)
        answer = llm_chain.invoke(question)
        # print(answer, '\n')
        # print("Answer done")
        # print('end IA : ' + str(datetime.now())) 
        return(answer)

In [ ]:
search = widgets.Text(placeholder='Recherche swisscovery')
button = widgets.Button(description="Go!")
output = widgets.Output()

display(search, button, output)

def on_button_clicked(b):
    with output:
        clear_output()
        print(f"Recherche lancée, veuillez patienter, l'IA est un peu lente... ")
        recherche = search.value
        # print('ta recherche  = ' + myquery)
        answer = chat(recherche)

        # extraction de la query
        query = answer['text']
        query = query.replace('`', '')
        query = query.replace('´', '')
        if (query[0] == '\''):
            query = query[1:]
        if (query[-1] == '\''):
            query = query[:-1]
        if (query[0] == ' '):
            query = query[1:]
        if (query[-1] == ' '):
            query = query[:-1]
        
        # remplacement des retours de ligne
        query = query.replace('[explanation]', ':::')
        query = query.replace('[query]', '|||')
        query = query.replace('\n', '£££')
        query = query + '_;_'

        if ('|||' in query) and (':::' in query) > 0:
            # print ("Séparateurs OK")
            pattern = r"^(.*)\|\|\|(.*)_;_"
            match = re.search(pattern, query)
            answer_prof = match.group(1)
            query_clean = match.group(2)
        
        else :
            print ('ERROR, la réponse est mal formatée ! (c\'est ça un Hackathon....)')

        # nettoyage des deux parties
        answer_prof = answer_prof.replace(':::' , '')
        answer_prof = answer_prof.replace('£££' , ' ')
        answer_prof = answer_prof.replace('  ' , ' ')
        answer_prof = answer_prof.replace('  ' , ' ')
        answer_prof = answer_prof.replace(':::' , '')
        answer_prof = answer_prof.strip()
        print('')
        print('### Réponse de l\'IA : ')
        print(answer_prof)
        print('')
        print('')
        print ('### Vous devriez vérifier cette réponse en consultant les documents de notre collection : ')
        print ('')
        print ('')

        query_clean = query_clean.replace(':::' , '')
        if ('£££' in query_clean) :
            query_clean = query_clean[:query_clean.find('£££')]
        query_clean = query_clean.strip()
        # print(query_clean)

        # convertir en URL
        query_url = swisscovery_api + '&q=any,contains,' + urllib.parse.quote_plus(query_clean) + '&apikey=' + swisscovery_key
        # print(query_url)

        # requête sur l'API
        r = requests.get(query_url)
        # print (r.json())

        # nombre de résultats
        sw_json = r.json()
        sw_total = str(sw_json['info']['total'])
        
        # Affichage des résultats
        mylist = ''
        docs = sw_json['docs']
        for doc in docs :
            mylist = mylist + doc['pnx']['display']['title'][0]
            if ('contributor' in doc['pnx']['display']) : 
                mylist = mylist + '. - '
                i = 0
                for contributor in doc['pnx']['display']['contributor']:
                    if i < 3 :
                        mylist = mylist + contributor[:contributor.find('$')] + ' ; '
                    i = i + 1
            if ('ispartof' in doc['pnx']['display']) : 
                mylist = mylist  + doc['pnx']['display']['ispartof'][0][:doc['pnx']['display']['ispartof'][0].find('$')] + ', '
            if ('date' in doc['pnx']['display']) : 
                mylist = mylist + doc['pnx']['display']['date'][0] + ', '
            if ('mms' in doc['pnx']['display']) : 
                mylist = mylist + ' URL : https://slsp-unige.primo.exlibrisgroup.com/discovery/search?tab=41SLSP_UGE_MyInst_CI&search_scope=MyInst_and_CI&vid=41SLSP_UGE:VU1&offset=0&query=any,contains,' + doc['pnx']['display']['mms'][0]
            mylist = mylist + '\n\n'
        print (mylist)
        print ('')
        print ('Lancer la recherche créée par l\'IA sur swisscovery : https://slsp-unige.primo.exlibrisgroup.com/discovery/search?tab=41SLSP_UGE_MyInst_CI&search_scope=MyInst_and_CI&vid=41SLSP_UGE:VU1&offset=0&query=any,contains,' + urllib.parse.quote_plus(query_clean))
        print ('')
       
button.on_click(on_button_clicked)